In [2]:
def standardize_eeg_structure(eeg_data):
    """
    Standardizes EEG data structure (e.g., downsampling, renaming channels).
    
    Args:
    - eeg_data (np.array): Raw EEG data.
    
    Returns:
    - standardized_eeg (np.array): Standardized EEG data.
    """
    # Add your processing steps here (e.g., filtering, downsampling)
    # This is a placeholder example
    standardized_eeg = eeg_data  # You can add actual processing logic here
    return standardized_eeg


In [1]:
def get_pt_from_fname(filepath):
    """
    Extract patient ID from filename.
    
    Args:
    - filepath (str): Path to the file.
    
    Returns:
    - patient_id (str): Extracted patient ID.
    """
    # Extract the filename without extension
    import os
    import scipy.io
    import numpy as np

    filename = os.path.basename(filepath)
    
    # Assuming the patient ID is the second part of the filename, split by '_'
    parts = filename.split('_')
    
    if len(parts) > 1:
        patient_id = parts[1]  # Extracts '0284' from 'ICARE_0284_05'
    else:
        raise ValueError("Filename format is incorrect.")
    
    return patient_id


In [2]:
import wfdb

def extract_header(filename):
    record = wfdb.rdheader("0284_005_008_EEG")
    fs = record.fs
    channel = record.sig_name
    return fs, channel

In [3]:
def save_eeg_data(eeg_data, save_path):
    """
    Saves EEG data to a file.
    
    Args:
    - eeg_data (np.array): Processed EEG data.
    - save_path (str): Path to save the data.
    """
    import numpy as np
    
    np.save(save_path, eeg_data)


In [5]:
def reorder_eeg_channels(eeg_data, channel_names):
    import numpy as np

    desired_channel_order = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2',
                             'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']

    mapping_dict = {}
    reordered_channels_data = []

    patient_eeg = eeg_data  # Shape: [n_channels, n_samples]
    patient_channels = channel_names  # List or array of channel names

    n_samples = patient_eeg.shape[1]  # Number of samples per channel

    for channel in desired_channel_order:
        if channel in patient_channels:
            index = patient_channels.index(channel)
            mapping_dict[channel] = index
            channel_data = patient_eeg[index]  # Extract the channel data
            reordered_channels_data.append(channel_data)
        else:
            # If the desired channel is not present, fill with zeros
            mapping_dict[channel] = None
            channel_data = np.zeros(n_samples)
            reordered_channels_data.append(channel_data)
            print(str(channel) + " is missing, so it was padded with zeros")

    # Convert the list to a NumPy array
    reordered_eeg_data = np.array(reordered_channels_data)

    return reordered_eeg_data, mapping_dict


In [31]:
import numpy as np
from scipy.signal import resample
import resampy

def standardize_struct(input_data, fs, channel):
    """
    Standardize EEG data structure by unifying channel nomenclature, reordering channels, and downsampling.
    
    Args:
    - input_data (dict): Input EEG data in a dictionary format with keys 'header' and 'matrix'.
    
    Returns:
    - output (dict): Standardized EEG data structure.
    """

    # Constants
    FREQUENCY = fs  # Assumes same frequency for all channels
    N_CHANNELS = 19  # Final number of channels
    TARGET_FREQUENCY = 200  # Target frequency for downsampling

    # Unify nomenclature and reorder channels
    # Assuming `map_channels` is a function that maps input channels to the standard ones
    reordered_eeg, channel_mapping = reorder_eeg_channels(input_data, channel)  # Get channel mapping
    num_chans = len(input_data)

    # Initialize output structure
    # output = {
    #     'header': {
    #         'ns': N_CHANNELS,
    #         'label': ['Fp1', 'Fp2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'O1', 'O2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'Fz', 'Cz', 'Pz'],
    #         'transducer': [None] * N_CHANNELS,
    #         'units': [None] * N_CHANNELS,
    #         'physicalMin': np.zeros(N_CHANNELS),
    #         'physicalMax': np.zeros(N_CHANNELS),
    #         'digitalMin': np.zeros(N_CHANNELS),
    #         'digitalMax': np.zeros(N_CHANNELS),
    #         'prefilter': [None] * N_CHANNELS,
    #         'frequency': [TARGET_FREQUENCY] * N_CHANNELS
    #     },
    #     'matrix': np.zeros((N_CHANNELS, resample(input_data['matrix'][0], TARGET_FREQUENCY, FREQUENCY).shape[0]))
    # }

    # # Downsample and reorder channels
    # for k in range(num_chans):
    #     if not np.isnan(s_to_m[k]):  # Check if there is a mapping for the current channel
    #         # Downsample the current channel
    #         output['matrix'][s_to_m[k]] = resample(input_data['matrix'][k], TARGET_FREQUENCY, FREQUENCY)

    #         # Copy header information for each mapped channel
    #         output['header']['transducer'][s_to_m[k]] = input_data['header']['transducer'][k]
    #         output['header']['units'][s_to_m[k]] = input_data['header']['units'][k]
    #         output['header']['physicalMin'][s_to_m[k]] = input_data['header']['physicalMin'][k]
    #         output['header']['physicalMax'][s_to_m[k]] = input_data['header']['physicalMax'][k]
    #         output['header']['digitalMin'][s_to_m[k]] = input_data['header']['digitalMin'][k]
    #         output['header']['digitalMax'][s_to_m[k]] = input_data['header']['digitalMax'][k]
    #         output['header']['prefilter'][s_to_m[k]] = input_data['header']['prefilter'][k]

    output = resampy.resample(reordered_eeg, TARGET_FREQUENCY, FREQUENCY)

    return output, channel_mapping

In [27]:
import os
import numpy as np
import scipy.io

def load_mat_eeg(fs, channel_order, filepath, use_saved=True, save_result=True):
    """
    Load EEG data from .mat files, process it, and optionally save/load intermediate results.
    
    Args:
    - filepath (str): Path to the .mat file.
    - use_saved (bool): If True, loads a saved processed EEG file if it exists.
    - save_result (bool): If True, saves the processed EEG data to a .npy file.
    
    Returns:
    - eeg_data (np.array): Processed EEG data.
    """
    filename_no_ext, ext = os.path.splitext(filepath)
    patient_id = get_pt_from_fname(filepath)  # Adjust as per your function to extract '0284'

    # Define save path for the processed EEG data
    eeglab_save_folder = os.path.join('output_dir', 'eeglab', patient_id)
    eeglab_save_path = os.path.join(eeglab_save_folder, f"{filename_no_ext}_eeglab.npy")
    
    # Create directory if it doesn't exist
    if not os.path.exists(eeglab_save_folder):
        os.makedirs(eeglab_save_folder)

    # Load saved EEG data if it exists and use_saved is True
    if use_saved and os.path.exists(eeglab_save_path):
        print("EEGLAB file already exists, loading...")
        eeg_data = np.load(eeglab_save_path)
        if check_loaded_eeglab(eeg_data):  # Check if the loaded data is valid
            return eeg_data
    
    # Process and load .mat file if saved file does not exist
    print(f"Processing .mat file from {filepath}")
    eeg_struct = scipy.io.loadmat(filepath)
   
    # Extract EEG data from the .mat structure (adjust key based on your file structure)
    eeg_data = eeg_struct.get('val', None)  
    
    # if eeg_data is None:
    #     raise ValueError("EEG data not found in the .mat file.")
    
    # Standardize the EEG structure (if you have custom logic for standardization)
    eeg_data, channel_mapping = standardize_struct(eeg_data, fs, channel_order)
    
    print(channel_mapping)
    # Save the result if save_result is True
    if save_result:
        print(f"Saving processed EEG data to {eeglab_save_path}")
        np.save(eeglab_save_path, eeg_data)
    
    return eeg_data, channel_mapping

def check_loaded_eeglab(eeg_data):
    """
    Check if loaded EEG data is valid.
    
    Args:
    - eeg_data (np.array): Loaded EEG data.
    
    Returns:
    - is_good (bool): True if EEG data is valid, False otherwise.
    """
    if eeg_data is not None and len(eeg_data.shape) == 2:
        nchans, ns = eeg_data.shape
        if nchans > 0 and ns > 0:
            print("Loaded EEG data seems good.")
            return True
    print("Invalid EEG data.")
    return False

def check_loaded_eeglab(eeg_data):
    """
    Check if loaded EEG data is valid.
    
    Args:
    - eeg_data (np.array): Loaded EEG data.
    
    Returns:
    - is_good (bool): True if EEG data is valid, False otherwise.
    """
    if eeg_data is not None and len(eeg_data.shape) == 2:
        nchans, ns = eeg_data.shape
        if nchans > 0 and ns > 0:
            print("Loaded EEG data seems good.")
            return True
    print("Invalid EEG data.")
    return False


In [37]:
fs, channels = extract_header("0284_005_008_EEG")
fs_target = 200
print(fs, channels)

500 ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']


In [39]:
fname = r'0284_005_008_EEG.mat'
loaded_eeg, channel_names = load_mat_eeg(fs, channels, fname)

Processing .mat file from 0284_005_008_EEG.mat
{'Fp1': 0, 'Fp2': 1, 'F3': 2, 'F4': 3, 'C3': 4, 'C4': 5, 'P3': 6, 'P4': 7, 'O1': 8, 'O2': 9, 'F7': 10, 'F8': 11, 'T3': 12, 'T4': 13, 'T5': 14, 'T6': 15, 'Fz': 16, 'Cz': 17, 'Pz': 18}
Saving processed EEG data to output_dir\eeglab\005\0284_005_008_EEG_eeglab.npy


In [40]:
print(loaded_eeg)

[[  6763.219    8033.191    7943.328  ... -23666.023  -17614.791
   -8730.428 ]
 [  4041.3198   4543.7593   4195.2827 ... -27634.52   -20481.64
  -10103.961 ]
 [  8358.308   10200.477   10467.431  ... -24870.371  -18591.982
   -9210.098 ]
 ...
 [  9829.155   11167.829   10430.464  ... -24704.719  -18513.54
   -9216.05  ]
 [  9321.079   10845.343   10225.378  ... -23526.992  -17867.096
   -8986.43  ]
 [  5166.2188   5835.3296   5249.381  ... -23831.965  -17763.824
   -8756.343 ]]


In [35]:
import numpy as np
from scipy.signal import butter, filtfilt

def preprocess(eeg_data, srate):
    """
    Preprocess the EEG data by applying filters and converting to bipolar montage referencing.
    
    Args:
    - eeg_data (np.array): EEG data as a NumPy array (channels x samples).
    - srate (int): Sampling rate of the EEG data.
    
    Returns:
    - eeg_preprocessed (np.array): Preprocessed EEG data.
    """

    # Get filter parameters (hardcoded for now, replace with your function if needed)
    hp_filter_order = 4  # High-pass filter order
    low_freq_threshold = 0.5  # Low frequency threshold in Hz
    lp_filter_order = 4  # Low-pass filter order
    high_freq_threshold = 50  # High frequency threshold in Hz
    notch_filter_order = 4  # Notch filter order
    notch_low_threshold = 59  # Notch filter lower bound in Hz
    notch_high_threshold = 61  # Notch filter upper bound in Hz

    # Design filters
    bh, ah = butter(hp_filter_order, low_freq_threshold / (srate / 2), 'high')  # High-pass filter
    bl, al = butter(lp_filter_order, high_freq_threshold / (srate / 2), 'low')  # Low-pass filter
    bn, an = butter(notch_filter_order, 
                    [notch_low_threshold / (srate / 2), notch_high_threshold / (srate / 2)], 
                    'bandstop')  # Notch filter

    # Transpose the data (scipy expects samples x channels format)
    data = eeg_data.T

    # Apply the filters using filtfilt (zero-phase filtering)
    data = filtfilt(bh, ah, data, axis=0)  # High-pass filter
    data = filtfilt(bn, an, data, axis=0)  # Notch filter
    data = filtfilt(bl, al, data, axis=0)  # Low-pass filter

    # Transpose back to channels x samples
    eeg_preprocessed = data.T

    # Apply bipolar montage referencing (this function would need to be defined based on your data)
    # eeg_preprocessed = get_bipolar_montage_EEGLab(eeg_preprocessed)

    return eeg_preprocessed

In [41]:
preprocessed_eeg = preprocess(loaded_eeg, fs_target)

In [42]:
print(preprocessed_eeg)

[[   -59.89382334   1081.43293431   1202.86200127 ... -16699.77722144
  -11471.2959849   -3132.67453937]
 [  -364.94276011     48.17688401   -142.57607303 ... -19490.47089794
  -13297.31907867  -3538.98687612]
 [   524.33573128   2216.79196948   2712.73582578 ... -17397.4578139
  -11909.67542229  -3018.06947636]
 ...
 [   118.97649709   1201.81816706    815.75763665 ... -17105.71536667
  -11710.6940293   -2933.16583144]
 [   375.61989344   1666.49479921   1361.59066278 ... -15916.37900994
  -10940.42097093  -2507.80162213]
 [   364.77923099    907.37272904    513.55737731 ... -16614.09289783
  -11298.46429115  -2736.60596527]]
